# Pre-trained deep neural networks in PyTorch

**Objectives**

This week, we will apply what we learned in the tutorials and get a quick idea of what a deep neural network is capable of when it comes to image classification tasks. To do so, we will play with a pre-trained neural network (ResNet101). 

## Contents:

1. Pre-trained deep neural networks in PyTorch
2. Making predictions using a neural network in Pytorch  
    1. Defining a preprocess pipeline using PyTorch's transforms  
    2. Loading and preprocessing data  
    3. Making predictions using our neural network  
    4. Interpreting the output  
3. Playing with the ResNet model
4. Good to know

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchvision import transforms
from PIL import Image
from os import listdir

## 1. Pre-trained deep neural networks in PyTorch

As written in the documentation:

> The [torchvision.models](https://pytorch.org/vision/stable/models.html#torchvision-models) subpackage contains definitions of models for addressing different tasks, including: image classification, pixelwise semantic segmentation, object detection, instance segmentation, person keypoint detection and video classification. \[...\] It provides pre-trained models.

[ResNet](https://pytorch.org/vision/stable/models.html#id10) is a deep residual neural network that aims at classifying images. In Pytorch, several pre-trained ResNet models are available with different depths (resnet18, resnet34, resnet50, resnet101 and resnet152). Here we will use [resnet101](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet101).

These pre-trained models were built and trained exactly as we did with our custom neural networks in the tutorials and can also be used in the exact same way. Unsurprisingly, they also subclass [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module).

In [ ]:
# The next line is all we need to create an instance of a pre-trained ResNet101 model 
# 101 means that we choose the ResNet architecture with 101 layers
resnet = models.resnet101(pretrained=True)   
print("Pytorch class of pre-trained  models: ", type(resnet))
print("Which is subclass of a nn.Module:     ", issubclass(type(resnet), nn.Module))
print("\n", resnet)      

**QUESTIONS**

1. If we have 1000 different labels (e.g cat, dog, mouse, goose, etc) what should be the dimension of the output layer of the neural network?
1. In the output above we can see a module called "Sequential". We already met this module in the second and third tutorial, can you briefly explained what it is?
1. In the output above we can also see a module called "Bottleneck". This module was very quickly mentioned in the third tutorial, do you remember what it is? 

## 2. Making predictions using a neural network in Pytorch

In this section we will:

1. Load an image and our labels
1. Preprocess our image
1. Make predictions using our neural network
1. Interpret the output

### 2.1 Defining a preprocess pipeline using PyTorch's transforms

As we saw in the tutorials, the [torchvision.transforms](https://pytorch.org/vision/stable/transforms.html#torchvision-transforms) module can easily performs the most common image transformations such as [Resize](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Resize), [CenterCrop](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.CenterCrop), [ToTensor](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor), [Normalize](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Normalize), etc. In addition, this module allows us to quickly define preprocessing pipelines using the [transforms.Compose](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Compose) method.

In the following cell we define the pre-processing transformations that will be applied on our input images. Remember that when it comes to storing numerical data, the "PyTorch-friendly objects" are not numpy arrays but PyTorch's [tensors](https://pytorch.org/docs/stable/tensors.html#torch.Tensor) and that the [ToTensor](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.ToTensor) transform implicitly:

1. Reshapes a ``(H, W, C)`` image into a ``(C, H, W)`` tensor (Height, Width, Channel (color))
2. Rescales ``[0 255]`` int arrays into ``[0 1]`` float tensors

**TODO** 

Use [transforms.Compose](https://pytorch.org/vision/stable/transforms.html#torchvision.transforms.Compose) as well as appropriate transforms in order to define a preprocessor ``preprocessor`` that:
1. Resize images to ``256x256``  
1. Crop images, keeping only the ``224x224`` pixels at the center
1. Transform images to tensors
1. Normalize tensors, using ``mean = [0.485, 0.456, 0.406]`` and ``std = [0.229, 0.224, 0.225]``

In [ ]:
#TODO:
preprocessor = ...

### 2.2 Loading and preprocessing data

In [ ]:
# ------------------------------
# Images
# ------------------------------

# Load one of our images
img = Image.open("imgs/Bobby.jpeg")
# Preprocess our image using our preprocessor ('t' stands for 'tensor')
img_t = preprocessor(img)
# Reshape so that it is a batch (of size 1) as required in Pytorch         
batch_t = torch.unsqueeze(img_t, 0)
# Check that it has the required shape (N, C, H, W)
# (See 2nd tutorial if you're struggling with shape conventions in Pytorch)
print("Shape of our input batch: ", batch_t.size())

# ------------------------------
# Labels
# ------------------------------

# Read all the labels with which ResNet was trained and store them in the list 'labels'
with open('list_labels.txt') as f:
    labels = [line.strip() for line in f.readlines()]

### 2.3 Making predictions using our neural network

After recalling that: 

> "Some models use modules which have different training and evaluation behavior, such as batch normalization. To switch between these modes, use [model.train()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.train) or [model.eval()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.eval) (from the [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module)) as appropriate.

We are now ready to make some predictions on our images. Let's show the output of the resnet model given our image of Bobby the Golden Retriever.

**QUESTION** 

1. Set ``resnet`` in evaluation mode.
1. Compute the output ``out`` corresponding to the input batch ``batch_t`` (defined in the cell above) 
1. Print the output tensor
1. Print the dimension the output tensor using the [Tensor.size()](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.size) method
2. Does it match your previous answer about the output dimension? 

In [ ]:
# TODO:

### 2.4 Interpreting the output

You don't know what to do with that tensor right? How do you know if this output tensor means that the image is a dog or a cat or something else? 

Well that's actually simple. The first idea would be to find the most activated output unit, that is to say, the index of max value and find the label with the corresponding index. To do so we use the [torch.max](https://pytorch.org/docs/stable/generated/torch.max.html?highlight=max#torch.max) function

In [ ]:
_, index = torch.max(out, 1)
print(
    "Index: ", index,  
    "\nLabel: ", labels[index], 
    "\nOutput value: ", out[0, index]
    ) 

Now the question is "how to interpret this output value?" How can we say if the model hesitates between this label and another one? 

We would like to convert this tensor value into something that could be interpreted as the confidence that the model has in its prediction. To do so, we use the [softmax](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.softmax) function which normalizes our outputs to \[0, 1\]
For more information about the SoftMax function, you can watch the videos by Andrew Ng: 
- [Softmax Regression (C2W3L08)](https://www.youtube.com/watch?v=LLux1SW--oM)
- [Training Softmax Classifier (C2W3L09)](https://www.youtube.com/watch?v=ueO_Ph0Pyqk)

**QUESTION** 

1. Find the index corresponding to the max value of ``out`` **Hint:** Look at the previous cell 

In [ ]:
# TODO: Find the index corresponding to the max value of out
_, index = ...
confidences = F.softmax(out, dim=1)[0]
percentages = confidences * 100
print(
    "Label: ",labels[index[0]], 
    "\nConfidence: ", round(percentages[index[0]].item(), 2), "%")

#### Top-1 and Top-5 errors

When evaluating an image classifier we often use the terms *Top-1 error* and *Top-5 error* 

If the classifier’s top guess is the correct answer (e.g., the highest score is for the “dog” class, and the test image is actually of a dog), then the correct answer is said to be in the Top-1. If the correct answer is at least among the classifier’s top 5 guesses, it is said to be in the Top-5.

The top-1 score is the conventional accuracy, that is to say it checks if the top class (the one having the highest confidence) is the same as the target label. This is what we have done in the cell above. On the other hand, the top-5 score checks if the target label is one of your top 5 predictions (the 5 ones with the highest confidences). To do so we use the [torch.sort](https://pytorch.org/docs/stable/generated/torch.sort.html#torch-sort) function

**QUESTIONS**

1. Complete the code below **Hint:** Look at how we preprocessed the first image Bobby 
2. Does the model seem confident about the first prediction?

In [ ]:
num_preds = 5

img = Image.open("imgs/golden_retriever_online.jpeg")
# TODO: preprocess the image 
img_t = ...
# TODO: create a batch of size 1
batch_t = ...
# TODO: Compute the output tensor of the tensor image contained in img_t
out = ...
# TODO: Compute the percentage representing the confidence of the model about the output
percentages = ...
_, indices = torch.sort(out, descending=True)

results = [(labels[idx], round(percentages[idx].item(), 2)) for idx in indices[0][:num_preds]]
for i_pred in range(num_preds):
    print(
        "Guess number ", i_pred, ": ",
        "\n    Label: ", results[i_pred][0], 
        "\n    Confidence: ",  results[i_pred][1],"%"
        )

## 3. Playing with the ResNet model

Put all the images that you want in the 'imgs/' folder (could be personal pictures or taken from the internet)

**QUESTIONS**

1. Complete the code below so that for each image it prints the 5 best guests according to the model
2. When the image is a dog, what are usually the 1st, 2nd, 3rd guesses? 
3. Use one of your personal pictures of an object whose label is in the list of labels.
4. Try to find an image on the web whose label is in the list of labels but whose corresponding prediction is wrong. How can you try to make it difficult for the model to recognize the object? 
5. Try to find an image on the web whose label is NOT in the list of labels with which the model was trained. Look at the output, is it consistent even though it is necessarily wrong? 

In [ ]:
# ------------------------------
# Load inputs
# ------------------------------

# Load all the images in the 'imgs/' folder
list_img_t = []                  # Where input tensors will be stored
path_imgs = 'imgs/'   
list_files = listdir('imgs/')    # Find all filenames in the 'imgs/' folder
for f in list_files:
    img = Image.open(path_imgs + f)
    img = img.convert('RGB')  # Because some of the images are in the RGBA format while ResNet requires a RGB format
    # TODO: preprocess the image
    ...
    list_img_t.append(torch.unsqueeze(img_t, 0) )

# ------------------------------
# Make predictions
# ------------------------------
num_preds = 5
for i, batch_t in enumerate(list_img_t):
    print("\n ====== ", list_files[i], " ====== ")

    # TODO: Compute the output tensor of the tensor image contained in batch_t
    ...
    # TODO: Compute the percentage representing the confidence of the model about the output
    ...
    # TODO: Sort the out tensor in descending order
    _, indices = ...
    results = [(labels[idx], round(percentages[idx].item(), 2)) for idx in indices[0][:num_preds]]
    for i_pred in range(num_preds):
        print(
            "Guess number ", i_pred, ": ",
            "\n    Label: ", results[i_pred][0], 
            "\n    Confidence: ",  results[i_pred][1],"%"
            )

## 4. Good to know
- In PyTorch, data are stored in [tensors](https://pytorch.org/docs/stable/tensors.html#torch.Tensor). This is the Pytorch counterpart of Numpy's array and most of the methods that are available in Numpy are also available in Pytorch. (e.g 
[size](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.size), 
[amax](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.amax), 
[argmax](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.argmax), 
[sort](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.sort), 
[abs](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.abs), 
[cos](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.cos), 
[sum](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.sum) etc.)
- In PyTorch all neural networks should be a class that is itself a subclass of the PyTorch's [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#module) class
- There are many well-known deep neural network architectures available in the [torchvision.models](https://pytorch.org/vision/stable/models.html?highlight=models) sub-package. 
  - For each of these architectures a pre-trained model is available. 
  - Some of them such as the ResNet architecture even have multiple pre-trained model instances of different depths. For the [ResNet](https://pytorch.org/vision/stable/models.html#id10) class, we have [resnet18](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet18), [resnet50](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet50), [resnet101](https://pytorch.org/vision/stable/models.html#torchvision.models.resnet101), etc.
- During the preprocessing, we can use the [torchvision.transforms](https://pytorch.org/vision/stable/transforms.html#torchvision-transforms) module to perform the most common image transformations
- Some models use modules that have different training and evaluation behavior, such as batch normalization. To switch between these modes, we use [model.train()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.train) and [model.eval()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.eval) accordingly
- Top-1 and Top-5 scores are commonly used in image classification
- When there are more than 2 possible classes we often use the [SoftMax]((https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.softmax)) function in the output layer to convert the output tensor values into confidence values.
- However, we will see in this course that we don't need a softmax function in the output layer if we use [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentro#torch.nn.CrossEntropyLoss) loss function.